# RAS ile ROS ÖĞRENİYORUM

## Ünite 2: ROS Topics

Bu Eğitimin verimli olması için python ve linux'u temel seviyede kullanabilir olmanız gerekmektedir.

Bu ünitede şunları göreceksiniz:
1- ROS içinde tanımlanan temel kavramlar olan Node, Package, Topic, Topics Messages kavramları ve bunların nasıl kullanıldığı 
2- Publisher ve Subscriber kavramları 
3- Simulasyon robotu: Turtlebot 3

**Part 1: Publishers**

Öncelikle terminali açarak şu komutunu çalıştırıyoruz ve ROS'u başlatıyoruz. Buradan sonraki adımlarda ilk terminalimizde roscore çalışıyor durumda olacak ve yeni terminaller açacağız.

In [ ]:
~$ roscore

Daha sonra ilk python dosyamızı oluşturmak üzere /catkin_ws workspace içerisindeki /src dosyasına terminalimizi yönlendiriyor ve bu konumda çalışabilir durumdaki ilk python dosyamızı oluşturuyoruz.

In [ ]:
~$ cd catkin_ws/src
~/catkin_ws/src$ touch simple_topic_publisher.py
~/catkin_ws/src$ chmod +x simple_topic_publisher.py

Ardından bu dosyayı düzenlemek için terminale şunu yazıyoruz ve altındaki python kodunu açılan yeni pencerede kopyalayıp yapıştırıyoruz.

In [ ]:
~/catkin_ws/src$ gedit simple_topic_publisher.py

In [ ]:
#! /usr/bin/env python
import rospy
from std_msgs.msg import Int32

rospy.init_node('topic_publisher')

pub = rospy.Publisher('/counter', Int32, queue_size=1)

rate = rospy.Rate(2)

count = Int32()
count.data = 0

while not rospy.is_shutdown():
    pub.publish(count)
    count.data += 1
    rate.sleep()

In [ ]:
import rospy 

Python ile çalıştığımızdan dolayı import edilmelidir.

from std_msgs.msg import Int32 

"/std_msgs" isimli bir pakette msg klasöründe Int32 isimli bir mesaj dosyası vardır. OLuşturduğumuz topic bu mesaj tipiyle yayın (publish) yapacaktır. Dolayısıyla bu mesaj tipinin import edilmesi gereklidir.

In [ ]:
rospy.init_node('topic_publisher')

"topic_publisher" adında bir node oluşturduk. Bu node üzerinden bir topic hakkında yayın yapacağız. Keyfi bi isim alabilir.

pub = rospy.Publisher('/counter', Int32, queue_size=1)

"/counter" adlı bir topic üzerinden Int32 mesaj tipinde yayın yapacağız.Bu topic ismi keyfi olabildiği gibi önceden tanımlanmış bir topic ismi de olabilir. "/cmd_vel", "/odom" gibi.


In [ ]:
rate = rospy.Rate(2)
rate.sleep()

Yapacağımız yayının frekansını belirtiriz. Burada 2 Hz olarak belirledik.

In [ ]:
count = Int32()
count.data = 0

Burada count adında bir değişken tanımlarız ve Int32 mesaj tipindedir.
"data" Int32 mesaj tipinin içinde yer alan bir parametredir ve tanımdalığımız değişkenin parametresi olarak ona değer veriyoruz.

In [ ]:
while not rospy.is_shutdown():
    pub.publish(count)
    count.data += 1
    
while döngüsü burada terminal üzerinde ctrl+c basılmadığı sürece çalışır durumda olacaktır. Döngü sırasında belli bir periyotta count değişkenine tanımladığımız parametrelerin hepsi (şuan sadece count) yayınlanacaktır. Her bir periyotta ayrıca bu parametre artacaktır.

Şimdi yazdığımız kodu çalıştırmaya geldi. Aşağıdaki kodu yazıyoruz.

In [ ]:
~/catkin_ws/src$ python simple_topic_publisher.py

Ee bir şey olmadı? Çünkü biz yayını yaptık ancak henüz bu yayına abone olan veya dinleyen birisi yok. Gelin yaptığımız yayını dinleyelim. Ayrı bir terminalde şunu yazalım:

In [ ]:
~/rostopic echo /counter

Tebrikler! İlk yayınınızı yapmış oldunuz ve ilk dinleyicisi de siz oldunuz. Gelin şimdi de tanımladığımız topic, node ve message kavramlarına terminalin gözünden bakalım.Aşağıdaki kodları teker teker yazarak inceleyelim.

In [ ]:
~$ rosnode list

Bu kodla şuan aktif olan node ları görebiliyoruz. Şimdi kendi tanımladığımız node için bir bakalım.


~$ rosnode info /topic_publisher

Burada şimdilik önemli olan "Publications" ve "Subscriptions" kısmıdır. "Publications" kısmı yayın yapılan "topic"leri,
"Subscriptions" da abone olunan "topic"leri göstermektedir.


~$ rostopic list

Bu kodla şuan aktif olan topic leri görebiliyoruz. Şimdi kendi tanımladığımız topic için bir bakalım.


~$ rostopic info /counter

Burada da bu topic üzerinden yayın yapan ve abone olan "node"ları görebiliyoruz. Ayrıca üstte de "topic"imizin mesaj tipini görebiliyoruz.


~$ roscd std_msgs/msg; ls

Burada standart mesaj(std_msgs) paketindeki mesaj tiplerini görebiliyoruz.


Son olarak da "topic"imizin mesaj tipini ve içindeki parametreleri inceleyelim.

~$ rostopic show std_msgs/Int32

Görüldüğü gibi Int32 mesaj dosyası içinde sadece data adlı parametre yer alıyor.

Ben başta simulasyon robotu felan gördüm, bu ne şimdi mi dediniz?
Sabredin bir sonraki bölümde Turtlebot ile tanışacağız ve onu
gazebo ortamında hareket ettireceğiz.

**Part 2 Turtlebot3 Robotunu Publisher kullanarak hareket ettirme**

Bu bölümde Turtlebot3 robotunu kullanarak publisher kavramının robotikte nasıl kullanıldığını tecrübe etmiş olacağız. 

Turtlebot3 robotu hakkında detaylı bilgi için:
https://emanual.robotis.com/docs/en/platform/turtlebot3/overview/

Turtlebot3 için Pc kurulumunu yapacağız. Bunu yaparken de aşağıdaki linki takip edeceğiz:
https://emanual.robotis.com/docs/en/platform/turtlebot3/pc_setup/#install-ubuntu-on-remote-pc

**Not: Turtlebot3 robotunu ROS melodic versiyonu için kurmaya çalışacağız. Kinetic versiyonu için aşağıda yapılan değişiklikleri dikkate almaya gerek yoktur.**

6.1.2'deki kodları birebir terminale kopyalayıp yapıştırarak çalıştırıyoruz. 6.1.3'e geçerken melodic versiyondan kaynaklı şu değişlikleri yapmamız gerekecek.

In [ ]:
~$ sudo apt-get install ros-melodic-joy ros-melodic-teleop-twist-joy ros-melodic-teleop-twist-keyboard ros-melodic-laser-proc ros-melodic-rgbd-launch ros-melodic-depthimage-to-laserscan ros-melodic-rosserial-arduino ros-melodic-rosserial-python ros-melodic-rosserial-server ros-melodic-rosserial-client ros-melodic-rosserial-msgs ros-melodic-amcl ros-melodic-map-server ros-melodic-move-base ros-melodic-urdf ros-melodic-xacro ros-melodic-compressed-image-transport ros-melodic-rqt-image-view ros-melodic-gmapping ros-melodic-navigation ros-melodic-interactive-markers

In [ ]:
$~ cd ~/catkin_ws/src/
~/catkin_ws/src$ git clone https://github.com/ROBOTIS-GIT/turtlebot3_msgs.git
~/catkin_ws/src$ git clone -b melodic-devel https://github.com/ROBOTIS-GIT/turtlebot3.git
~/catkin_ws/src$ cd ~/catkin_ws && catkin_make

Kurulumun hatasız tamamlanmış olması gerekiyor.
Robotu çalıştırmadan evvel güncelleme yapmamız gereken bir takım dosyalar var:

In [ ]:
~$ gedit .ignition/fuel/config.yaml

In [ ]:
url: https://api.ignitionfuel.org 
yerine aşağıdaki güncel adresi yazıyoruz.
url: https://api.ignitionrobotics.org

Ayrıca export etmek istediğimiz Turtlebot3 tipini belirliyoruz. .bashrc dosyasının içine aşağıdaki kodu ekliyor ve kaydediyoruz.

In [ ]:
~$ gedit .bashrc

In [ ]:
export TURTLEBOT3_MODEL=burger

Robot kurulumunun tamamlanmış olması gerekiyor. Robotumuzu görmek için Gazebo'yu açmamız gerekiyor. Şimdi yazacağımız kodla robotla beraber gazebo'yu açmış olacağız.

In [ ]:
~$ roslaunch turtlebot3_gazebo turtlebot3_empty_world.launch

Robotu öncelikle klavyeden manuel olarak kontrol edelim. Bunun için:

In [ ]:
~$ roslaunch turtlebot3_teleop turtlebot3_teleop_key.launch 

a,s,d,w,x tuşlarıyla robotu hareket ettirebilirsiniz. Sıkılmışsınızdır biraz, haydi deneyin!

Keyfiniz yerine geldiyse gelin işin sıkıcı kısmı publisher konusuna geri dönelim. 

Bunun için öncelikle bir paket oluşturalım. Adı da "my_package" olsun. Python ile yazacağımızdan "rospy" dependency olarak eklenmesi gerekiyor.

In [ ]:
~$ cd catkin_ws/src; 
~/catkin_ws/src$ catkin_create_pkg my_package rospy

In [ ]:
~/catkin_ws/src$ rospack profile

Ardından python dosyamızı ve launch dosyamızı sırasıyla oluşturacağız. 

In [ ]:
~/catkin_ws/src$ cd my_package/src
~/catkin_ws/src/my_package/src$ touch robot_mover.py
~/catkin_ws/src/my_package/src$ chmod +x robot_mover.py
~/catkin_ws/src/my_package/src$ cd ..
~/catkin_ws/src/my_package/$ mkdir launch; cd launch
~/catkin_ws/src/my_package/launch$ touch robot_launcher.launch

Dosyalarımızı oluşturduktan sonra aşağıdaki kodları sırasıyla python ve launch dosyalarına yapıştırıyoruz. Dosya değiştirme işlemlerini IDE üzerinden de kolaylıkla yapabilirsiniz. Dosyayı değiştirdikten kaydetmeyi unutmayın.

In [ ]:
#! /usr/bin/env python

import rospy
from geometry_msgs.msg import Twist

rospy.init_node('robot_mover_node')
pub=rospy.Publisher('/cmd_vel',Twist,queue_size=1)

rate=rospy.Rate(2)

move=Twist()
move.linear.x= 0.5 # [m/s]
move.angular.z= 0.5  # [rad/s]
while not rospy.is_shutdown():
    pub.publish(move)
    print("Robot moving...")
    rate.sleep()

In [ ]:
<launch> 
    <node pkg="my_package" type="robot_mover.py" name="robot_mover_node" output="screen" />
</launch>

Launch ve python dosyalarımız hazır. Geriye robotu çalıştırmak kalıyor. Aşağıdaki kodla beraber çalıştırarak robotumuza daire çizdirebiliriz.

In [ ]:
~$roslaunch my_package robot_launcher.launch

In [ ]:
Robotun doğrusal ve açısal hız değerleriyle oynarak farklı hareketler de elde edebilirsiniz.

Robotun hızını aynı zamanda terminalden de kontrol edebileceğinizi biliyor muydunuz? Terminali açarak şu kodları yazabilirsiniz:

In [ ]:
~$ rostopic pub /cmd_vel [TAB] [TAB]

In [ ]:
~$ rostopic pub /cmd_vel geometry_msgs/Twist "linear:
  x: 0.0
  y: 0.0
  z: 0.0
angular:
  x: 0.0
  y: 0.0
  z: 0.0"

Yukarıdaki ilk kodda topic adını girdikten sonra iki kere TAB'a basarak otomatik olarak topic mesaj tipine ve mesaj içeriğine ulaşmanızı sağlar. Örneğin robotu durdurmak için tüm değerleri sıfırlayarak topic'e yayın yaptırabilirsiniz.

Diyelim ki robot için bir hız verisi yayınlandınız ve terminali kapattınız. Robot hareket ediyor durumda kalsın ama siz robotun hızını unuttunuz. Robotun hız değerlerini almak için yaptığınız yayını dinlemelisiniz:

In [ ]:
~$ rostopic echo /cmd_vel

Ayrıca rostopic ile oluşturabileceğiniz diğer komutları görmek için aşağıdaki kodu terminale yazabilirsiniz:

In [ ]:
~$ rostopic [TAB] [TAB]

**Part 3 Subscribers**

Biz şu ana kadar publisher kullanarak yayın yapmayı, terminal üzerinden yayın yapmayı ve yine terminal üzerinden bu yayını dinlemeyi yaptık. Şimdi de subscriber kullanarak bu yayınları dinlemeyi deneyeceğiz. Kafanızda belki soru oluşmuş olabilir. Neden terminal üzerinden basitçe bu yayınlarla işlem yapmak varken bir sürü kod yazma uğraşına giriyoruz? Çünkü ileride bir robotla uğraşırken çok fazla sayıda yayın yapan node olacak. Örneğin bir yerden kamera verisi, bir yandan lazer, öbür yandan odometre ve hız gibi verilerle alakalı yayınlar ile uğraşacağız. Bu verileri işlemek ve aynı zamanda çoklu yayın yapma/dinleme kabiliyeti sağladığından publisher/subscriber ları kullanmak avantajlıdır.

Subscriber kullanma konusunda kafamızda soru işareti kalmadıysa öncelikle yine dosyalarımızı oluşturalım. 

In [ ]:
~$ roscd my_package/src
~$ touch robot_listener.py
~$ chmod +x robot_listener.py
~$ cd ..
~$ cd lauch; touch subscriber_launcher.launch

In [ ]:
Ardından python ve launch dosyalarımıza aşağıdaki kodları sırasıyla yapıştırıyoruz.

In [ ]:
#! /usr/bin/env python
import rospy
from geometry_msgs.msg import Twist

def vel_info(msg):
    print msg

rospy.init_node('listener_node')

sub = rospy.Subscriber('/cmd_vel',Twist, vel_info) #you should publish velocity data before run this module

rospy.spin()

In [ ]:
<launch>
    <node pkg="my_package" type="robot_listener.py" name="listener_node" output="screen" />
</launch>

In [ ]:
Yayınladığınız hız verisini subscriber aracılığıyla dinlemek için terminalden aşağıdaki kodu yazabilirsiniz

In [ ]:
~$ roslaunch my_package subscriber_launcher.launch

Şuanda terminal üzerinde bir önceki bölümde python koduyla yayınladığımız hız değerlerini okuyabiliyoruz. Pekala hız verisini zaten biz yayınlıyoruz ve biliyoruz. Arık bunu dinlemenin bir anlamı yok. Gelin şimdi de robotun kendi yayınladığı odometre verilerini dinleyelim. Bu verilere bakarak robotun hangi konumda nereye dönük olduğunu öğrenebileceğiz. Python kodunu düzenleyerek aşağıdaki hale getirebilir, ya da yeni bir python & launch dosyası yaratarak pratik yapabilirsiniz. 

In [ ]:
#! /usr/bin/env python
import rospy
from nav_msgs.msg import Odometry

def pose_info(msg):
    
    # print (msg) #This will print the whole Odometry message
    # print msg.header #This will print the header section of the Odometry message
    print msg.pose # #This will print the pose section of the Odometry message

rospy.init_node('listener_node')
sub = rospy.Subscriber('/odom', Odometry, pose_info)

rospy.spin()